In [1]:
### ------------ ###
### System Setup ###
### ------------ ###

# Import modules/methods
import pandas as pd

### ------------ ###
### Pipeline dev ###
### ------------ ###

# Read in data
redfin = pd.read_csv('https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/county_market_tracker.tsv000.gz', sep = '\t')
re_inv = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv')
re_hot = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Hotness/RDC_Inventory_Hotness_Metrics_County_History.csv')

# Drop last rows of Realtor.com data (junk lines; no real data)
re_inv.drop(re_inv.tail(1).index, inplace=True)
re_hot.drop(re_hot.tail(1).index, inplace=True)

# Fields to keep from Redfin
keep_redfin = ['period_begin', 'region', 'property_type', 'median_list_price', 'median_sale_price'
                , 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'avg_sale_to_list', 'price_drops', 'sold_above_list', 'median_dom']

# Fields to keep from Realtor.com inventory data
keep_re_inv = ['month_date_yyyymm', 'county_fips', 'county_name', 'median_listing_price', 'pending_listing_count', 'new_listing_count', 'total_listing_count'
                , 'price_reduced_count', 'median_days_on_market']

# Fields to keep from Realtor.com hotness data
keep_re_hot = ['month_date_yyyymm', 'county_fips', 'county_name', 'hotness_rank', 'hotness_score', 'supply_score'
                , 'demand_score', 'ldp_unique_viewers_per_property_mm']

# Drop unwanted fields
redfin = redfin[keep_redfin]
re_inv = re_inv[keep_re_inv]
re_hot = re_hot[keep_re_hot]

# Rename fields
rename_redfin = ['year_month', 'county_name', 'property_type', 'median_list_price', 'median_sale_price', 'homes_sold', 'pending_sales', 'new_listings'
                , 'total_inventory', 'avg_sale_to_list', 'price_drops', 'sold_above_list', 'median_dom']
rename_re_inv = ['year_month', 'county_fips', 'county_name', 'median_list_price', 'pending_sales', 'new_listings', 'total_inventory', 'price_drops', 'median_dom']
rename_re_hot = ['year_month', 'county_fips', 'county_name', 'hotness_rank', 'hotness_score', 'supply_score', 'demand_score', 'unq_viewers_mm']

redfin.columns = rename_redfin
re_inv.columns = rename_re_inv
re_hot.columns = rename_re_hot

# Clean up text fields
redfin['county_name'] = redfin['county_name'].str.upper()
re_inv['county_name'] = re_inv['county_name'].str.upper()
re_hot['county_name'] = re_hot['county_name'].str.upper()

redfin['property_type'] = redfin['property_type'].str.upper()

# Convert data types
redfin['year_month'] = pd.to_datetime(redfin['year_month'])
redfin['county_name'] = redfin['county_name'].astype('str')
redfin['property_type'] = redfin['property_type'].astype('str')

# Convert to Parquet
redfin.to_parquet('../data/redfin.parquet')
re_inv.to_parquet('../data/realtor_inventory.parquet')
re_hot.to_parquet('../data/realtor_hotness.parquet')

/var/folders/27/22xv7cnn5dv21ywhxhvgv7p40000gn/T/ipykernel_30330/4232216850.py:14: DtypeWarning: Columns (0,1,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  re_inv = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv')
/var/folders/27/22xv7cnn5dv21ywhxhvgv7p40000gn/T/ipykernel_30330/4232216850.py:15: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  re_hot = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Hotness/RDC_Inventory_Hotness_Metrics_County_History.csv')
/var/folders/27/22xv7cnn5dv21ywhxhvgv7p40000gn/T/ipykernel_30330/4232216850.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

ArrowInvalid: ("Could not convert '201607' with type str: tried to convert to int64", 'Conversion failed for column year_month with type object')

In [9]:
re_inv['year_month'] = pd.to_datetime(re_inv['year_month'].str.slice(0,4) + '-' + re_inv['year_month'].str.slice(4,6) + '-01')
re_hot['year_month'] = pd.to_datetime(re_hot['year_month'].str.slice(0,4) + '-' + re_hot['year_month'].str.slice(4,6) + '-01')

0               NaT
1               NaT
2               NaT
3               NaT
4               NaT
            ...    
213691   2016-07-01
213692   2016-07-01
213693   2016-07-01
213694   2016-07-01
213695   2016-07-01
Name: year_month, Length: 213696, dtype: datetime64[ns]

In [ ]:
redfin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587193 entries, 0 to 587192
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   year_month         587193 non-null  object 
 1   county_name        587193 non-null  object 
 2   property_type      587193 non-null  object 
 3   median_list_price  541242 non-null  float64
 4   median_sale_price  586483 non-null  float64
 5   homes_sold         586489 non-null  float64
 6   pending_sales      504553 non-null  float64
 7   new_listings       536283 non-null  float64
 8   total_inventory    558603 non-null  float64
 9   avg_sale_to_list   579073 non-null  float64
 10  price_drops        338852 non-null  float64
 11  sold_above_list    581534 non-null  float64
 12  median_dom         578081 non-null  float64
dtypes: float64(10), object(3)
memory usage: 58.2+ MB
